# <center>Cómo funciona</center>

Ciertas páginas web intentan restringir la posibilidad de ser visitadas por robots creando el contenido que muestran en el cliente de forma dinámica (por medio de código Javascript), como es el caso de [milanuncios.com](https://www.milanuncios.com/).

### Scraping estático en Python


In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.milanuncios.com/anuncios/masajes-relajantes-en-jerez.htm"

soup = BeautifulSoup(requests.get(url).content, "html.parser")
print(soup.find(class_="aditem-detail-title"))

None


Sin embargo, es muy fácil saltarse estas limitaciones...

### Scraping dinámico en Python

In [2]:
from pyvirtualdisplay import Display  # pip3 install pyvirtualdisplay
from selenium import webdriver        # pip3 install selenium
from bs4 import BeautifulSoup         # pip3 install bs4
import time

url = "https://www.milanuncios.com/anuncios/masajes-relajantes-en-jerez.htm"

display = Display(visible=0, size=(800, 600))
display.start()
browser = webdriver.Firefox()  # Necesitas geckodriver y Mozilla Firefox
browser.get(url)
time.sleep(.8)
soup = BeautifulSoup(browser.page_source, "html.parser")
print(soup.find(class_="aditem-detail-title"))
browser.quit()

<a class="aditem-detail-title" href="/masajistas/masajes-y-tratamientos-fisio-domicilio-246582919.htm" target="_blank"><b class="sub2">MASAJES</b> Y TRATAMIENTOS FISIO DOMICILIO</a>


________________________________________________

# <center>Documentación</center>

## Uso básico
Los 4 métodos principales para realizar la gran mayoría de consultas en la página de [milanuncios.com](https://www.milanuncios.com/) son los siguientes:

In [ ]:
from milanuncios import MilAnuncios

# Debemos entrar a MilAnuncios como contexto (con la sentencia with)
with MilAnuncios() as ma:

    # Obtener las familias de categorías de la página principal:
    print(ma.categories) 

    # Obtener las subcategorías de una categoría:
    print(ma.subcategories("servicios"))

    # Realizar una consulta (indicamos el número de páginas, por defecto 1 página):
    print(ma.search("gatos", pages=3)) # Devuelve un DataFrame de pandas

    # Realizar una búsqueda por categoría/subcategoría:
    print(ma.search_category("juegos", subcategory="videoconsolas").tail())

### Renovación de anuncios

In [ ]:
from milanuncios import MilAnunciosLoginError

# El parámetro delay indica los segundos que esperamos en cargar las páginas y antes de ejecutar las acciones.
# Por defecto es 1.5, si tienes problemas de ejecución prueba a aumentarlo
with MilAnuncios(delay=3) as ma:
    # Loguearte en milanuncios
    ma.login("tu_email@proveedor.com", "tu_contraseña")  # Si falla el login se levantará MilAnunciosLoginError
    assert ma.logged == True
    
    # Obtener tus anuncios
    ma.my_ads(dataframe=False) # Por defecto devuelve un DataFrame de pandas, pero así devuelve una lista
    
    # Renovar tus anuncios
        # Puedes hacerlo por títulos o por número de anuncios
        # El programa ignorará los anuncios que no se pueden renovar aún
    ma.renew_ads(title=["Título de mi anuncio", "Otro, da igual si es en minúscula o mayúscula"])  # Por nombre
    
    ma.renew_ads(number=3)  # Los primeros 3 que se puedan renovar de tu lista de anuncios
    
    # Este método devuelve el número de anuncios renovados

También puedes obtener tus anuncios o renovarlos sin pasar por el método `login()`, sólo tienes que proveer tu email y contraseña a los métodos `my_ads()` o `renew_ads()` como primeros dos parámetros:

In [ ]:
options = {}
#options = dict(debug=True)  # Si quieres mostrar en pantalla el proceso de renovación activa el debug

with MilAnuncios(**options) as ma:
    ma.my_ads("tu_email@proveedor.com", "tu_contraseña")
    
    # Si no pasas número ni lista de títulos se renovarán todos los anuncios:
    ma.renew_ads("tu_email@proveedor.com", "tu_contraseña")                 

___________________________________

## Uso avanzado
### Filtros

In [ ]:
ma = MilAnuncios()
ma.__enter__()  # Así también podemos entrar como contexto

# Podemos filtrar por regiones en ambos métodos:
print(ma.search("guitarra flamenca", region="cadiz").head())
# Para consultar todas las regiones posibles (provincias y comunidades autónomas):
print(ma.regions)

# Si quieres buscar sólo por regiones (https://www.milanuncios.com/anuncios-en-sevilla/)
print(ma.search("", region="sevilla").head())  # Cadena vacía en el parámetro query

# Podemos filtrar por oferta y demanda. Por defecto, los parámetros offer y demand equivalen a True:
print(ma.search_category("inmobiliaria", subcategory="alquiler de casas", offer=False))

ma.__exit__()   # No te olvides de salir o acumularás procesos de Firefox en memoria

### Cache
Si vas a realizar muchas consultas en una sola sesión, es conveniente cachear todas las subcategorías al instanciar el scraper. Tardará un tiempo en guardar en memoria todas las subcategorías pero luego el rendimiento aumentará considerablemente:

In [ ]:
from pprint import pprint

with MilAnuncios(init_cache=True) as ma:
    pprint(ma.cache["categories"])
    pprint(ma.cache["subcategories"])